In [ ]:
#     df_trait["PHENO"] = admix.data.quantile_normalize(df_trait["PHENO"])
#     df_trait.to_csv(
#         join(OUT_DIR, f"{trait}.pheno"), sep="\t", index=False, na_rep="NA"
#     )
# dict_df_covar = {
#     group: pd.read_csv(join(OUT_DIR, f"{group}.covar"), sep="\t")
#     for group in group_list
# }
# for trait in trait_list:
#     # load trait
#     col_covar = ["SEX", "AGE"] + [f"PC{i}" for i in range(1, 11)]
#     dict_df_trait = {
#         group: pd.read_csv(join(OUT_DIR, f"{group}.{trait}.pheno"), sep="\t")
#         for group in group_list
#     }

#     df_train = pd.merge(
#         dict_df_trait["eur_train"], dict_df_covar["eur_train"], on=["FID", "IID"]
#     )
#     model = sm.OLS(
#         df_train["PHENO"], sm.add_constant(df_train[col_covar].values), missing="drop"
#     ).fit()
#     model.save(join(OUT_DIR, f"eur_train.{trait}.model"))
#     # regress out
#     for group in group_list:
#         df_group = pd.merge(
#             dict_df_trait[group], dict_df_covar[group], on=["FID", "IID"]
#         )
#         df_group["PHENO"] -= model.predict(sm.add_constant(df_group[col_covar].values))
#         df_group[["FID", "IID", "PHENO"]].to_csv(
#             join(OUT_DIR, f"{group}.{trait}.residual_pheno"),
#             sep="\t",
#             index=False,
#             na_rep="NA",
#         )

# 3. Perform GWAS
```bash
for trait in height bmi cholesterol hdl_cholesterol ldl_direct; do
    qsub prepare-train-gwas.sh ${trait}
done
```

# 4. Format GWAS

In [ ]:
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import admix
import admix_prs
from os.path import join

In [ ]:
trait_list = ["height", "bmi", "cholesterol", "hdl_cholesterol", "ldl_direct"]

GWAS_DIR = "../../data/train_gwas"
for trait in trait_list:
    assoc = admix_prs.plink2_assoc_to_ldpred2(
        join(GWAS_DIR, trait, "assoc.PHENO.glm.linear")
    )
    assoc.to_csv(
        join(GWAS_DIR, trait, "assoc.ldpred2.tsv"), index=False, sep="\t"
    )